In [213]:
import random
import numpy as np

# Creates a decision tree
class DecisionTree:
    
    # Initializes the attributes of the DecisionTree class.
    def __init__(self, dataset):
        self.dataset = dataset
        self.number_of_features = 0
        self.random_seed = 0

        self.instances = []
        self.feature_indicies = []
        self.feature_instances = []
        self.optimal_feature_index = 0
        self.optimal_feature_instance = 0

        self.unique_labels = []

        self.feature = 0
        self.feature_to_split = 0

        self.possible_outcomes = []
        
        self.nodes = []
        self.current_node = []

        self.optimal_features = []


    # Populates self.feature_instances with arrays of unique instances of each feature in the node
    def get_features(self, node):
        # Determining number of features within a node
        labels = []
        for instance in node:
            labels.append(instance[-1])
        
        self.unique_labels = []
        for instance in node:
            if instance[-1] not in self.unique_labels:
                self.unique_labels.append(instance[-1])

        np_labels = np.array(labels)
        for label in self.unique_labels:
            # Using numpy to count the occurences of each label within the node
            self.number_of_features = np.count_nonzero(np_labels == label)
        
        feature = 0
        while feature < self.number_of_features:
            for instance in self.dataset:
                if instance[feature] not in self.instances:
                    self.instances.append(instance[feature])
            pair = [feature, self.instances]
            self.feature_instances.append(pair)
            feature += 1
            self.instances = []

        for feature_number in range(self.number_of_features):
            self.feature_indicies.append(feature_number)

        # 1. Get argument: how many features there are, random seed
        # 2. Based on the number of features, use an algo to identify the different instances of that feature
        # 3. Every time new node is called, randomize the feature and how it is split. Once used, remove from lists.

    # Calculates the entropy of the current node
    
    def calculate_entropy(self, current_node):
    
        entropy = 0

        # Create an array of the labels 
        labels = []
        for instance in current_node:
            labels.append(instance[-1])
        print(f"All classes in the dataset: {labels}")
        
        unique_labels = []
        for instance in current_node:
            if instance[-1] not in unique_labels:
                unique_labels.append(instance[-1])
        print(f"\nAll unique labels in the dataset: {unique_labels}")

        # 1. Get the count of one unique label in labels and pop it all out (for loop with length of unique labels array)
        # 2. Apply to the entropy formula 
        # 3. Repeat
        
        np_labels = np.array(labels)
        for label in unique_labels:
            # Using numpy to count the occurences of each label within the node
            count = np.count_nonzero(np_labels == label)
            entropy += -count / len(labels) * np.log(count / len(labels)) / np.log(len(unique_labels))
        

        print(f"\nCalculated entropy: {entropy}")

        return entropy

        

    def calculate_weighted_average_times_child(self, child_node_1, child_node_2):
        
        entropy = 0
        weighted_average_times_children = 0

        for child_node in [child_node_1, child_node_2]:
            unique_labels = []
            for label in child_node:
                if label not in unique_labels:
                    unique_labels.append(label[-1])
            
            np_labels = np.array(child_node)
            for label in unique_labels:
                count = np.count_nonzero(np_labels == label)
                entropy += -count / len(child_node) * np.log(count / len(child_node)) / np.log(len(unique_labels))
                weighted_average_times_children += count / len(child_node) * entropy
        
        return weighted_average_times_children
    
    def calculate_information_gain(self, current_node, child_node_1, child_node_2):
        watc = self.calculate_weighted_average_times_child(child_node_1, child_node_2)
        IG = self.calculate_entropy(current_node) - watc
        return IG
        
    def split_root_node(self, root_node, of_index, of_instance):

        child_node_1 = []
        child_node_2 = []
        
        for instance in root_node:
            if instance[of_index] == of_instance:
                child_node_1.append(instance)
            if instance[of_index] != of_instance:
                child_node_2.append(instance)

        print(f"\nOptimized Child Node 1 with a feature index of {of_index} and a feature instance of {of_instance}: {child_node_1}")
        print(f"Optimized Child Node 2 with a feature index of {of_index} and a feature instance of {of_instance}: {child_node_2}")

        return child_node_1, child_node_2

# Calculating with information gain
# 1. Split the data with each condition and check which conditioned resulted in the greatest decrease in entropy after splitting
#    (or, the greatest information gain).
# 2. Select that condition for the first split.
# 3. Split the remaining nodes with the remaining conditions

    # Determines the optimal condition to split the previous node such that there is the greatest possible information gain.
    # 1. Stop after 1 split.
    # 2. Sort through an array of key-value pairs of entropy sum and feature used in the split condition.
    # 3. The feature of the pair with the lowest entropy sum will be used in the following node split condition.
    def optimal_condition(self, current_node):
        for feature_index in self.feature_indicies:
            for feature_instance in self.feature_instances[feature_index][1]:

                new_node_1 = []
                new_node_2 = []
                
                for instance in current_node:
                    if instance[feature_index] == feature_instance:
                        new_node_1.append(instance)
                    if instance[feature_index] != feature_instance:
                        new_node_2.append(instance)

                print(f"\nNode 1 with a feature index of {feature_index} and a feature instance of {feature_instance}: {new_node_1}")
                print(f"Node 2 with a feature index of {feature_index} and a feature instance of {feature_instance}: {new_node_2}")

                IG = self.calculate_information_gain(current_node, new_node_1, new_node_2)

                triplet = [IG, feature_index, feature_instance]
                self.possible_outcomes.append(triplet)
                
                self.possible_outcomes = sorted(self.possible_outcomes, reverse = True)
                self.optimal_feature_index = self.possible_outcomes[0][1]
                self.optimal_feature_instance = self.possible_outcomes[0][2]
        
        
        print(f"\nAll possible outcomes of the split: {self.possible_outcomes}")
        print(f"Optimal feature index: {self.optimal_feature_index}")
        print(f"Optimal feature instance: {self.optimal_feature_instance}")

        pair = [self.optimal_feature_index, self.optimal_feature_instance]
        self.optimal_features.append(pair)

        print(f"\nOptimal features: {self.optimal_features}")
        
        return self.optimal_feature_index, self.optimal_feature_instance
    
    def optimal_child_condition(self, child_node):
        for feature_index in self.feature_indicies:
            for feature_instance in self.feature_instances[feature_index][1]:

                new_node_1 = []
                new_node_2 = []
                
                for instance in child_node:
                    if instance[feature_index] == feature_instance:
                        new_node_1.append(instance)
                    if instance[feature_index] != feature_instance:
                        new_node_2.append(instance)

                print(f"\nNode 1 with a feature index of {feature_index} and a feature instance of {feature_instance}: {new_node_1}")
                print(f"Node 2 with a feature index of {feature_index} and a feature instance of {feature_instance}: {new_node_2}")

                IG = self.calculate_information_gain(child_node, new_node_1, new_node_2)

                triplet = [IG, feature_index, feature_instance]
                self.possible_outcomes.append(triplet)
                
                self.possible_outcomes = sorted(self.possible_outcomes, reverse = True)
                self.optimal_feature_index = self.possible_outcomes[0][1]
                self.optimal_feature_instance = self.possible_outcomes[0][2]
        
        
        print(f"\nAll possible outcomes of the split: {self.possible_outcomes}")
        print(f"Optimal feature index: {self.optimal_feature_index}")
        print(f"Optimal feature instance: {self.optimal_feature_instance}")

        pair = [self.optimal_feature_index, self.optimal_feature_instance]
        self.optimal_features.append(pair)

        print(f"\nOptimal features: {self.optimal_features}")
        
        return self.optimal_feature_index, self.optimal_feature_instance

    # 0. Apply a stopping criteria
    # 1. Find optimal features for parent mode, split data with optimal features
    # 2. Find optimal features for the next child node, split data with optimal features
    # 3. Repeat

    def build_tree(self, root_node):
        self.get_features(root_node)
        of_index, of_instance = self.optimal_condition(root_node)
        child_node_1, child_node_2 = self.split_root_node(root_node, of_index, of_instance)
        
        self.build_children(child_node_1, child_node_2)
    
    def build_children(self, child_node_1, child_node_2):

        for node in [child_node_1, child_node_2]:
            self.get_features(node)
            if len(self.unique_labels) == 1:
                self.optimal_features.append([-1, node[-1][-1]])
                pass
            else:
                of_index, of_instance = self.optimal_child_condition(node)
                child_node_1, child_node_2 = self.split_root_node(node, of_index, of_instance)
                self.build_children(child_node_1, child_node_2)
    
    # Using the optimal features, construct a data tree that results in the unclassified instance being sorted into a leaf node.

    # 1. If-else statements to traverse the data point through the trained tree.
    # 2. If it reaches a leaf node, classify as the classes from that leaf node

    # Traversing the data
    # 1. Iterate through the optimal features from the trained dataset
    # 2. If following value is -1, print class if passes condition

    def classify_data_point(self, data_point):
        for feature in self.optimal_features:
            self.next_index = self.optimal_features.index(feature) + 1
            if self.next_index == len(self.optimal_features):
                break
            if data_point[feature[0]] == feature[1]:
                if self.optimal_features[self.next_index][0] == -1:
                    print(f"Class {self.optimal_features[self.next_index][-1]}")
                    break
            else:
                pass


In [215]:
practice_training_dataset_1 = [[0, 3, 0], 
                             [1, 3, 1], 
                             [2, 1, 2],  
                             [0, 3, 0], 
                             [1, 3, 1], 
                             [2, 1, 2]]

# Training the model
train_rf = DecisionTree(practice_training_dataset_1)
print(f"Node: {practice_training_dataset_1}")
train_rf.build_tree(practice_training_dataset_1)
print(f"\nOptimal features: {train_rf.optimal_features}")

# Verifying the model's accuracy
practice_unclassified_instance = [0, 3]
train_rf.classify_data_point(practice_unclassified_instance)

practice_unclassified_instance = [1, 3]
train_rf.classify_data_point(practice_unclassified_instance)

practice_unclassified_instance = [2, 1]
train_rf.classify_data_point(practice_unclassified_instance)

practice_unclassified_instance = [0, 3]
train_rf.classify_data_point(practice_unclassified_instance)

practice_unclassified_instance = [1, 3]
train_rf.classify_data_point(practice_unclassified_instance)

practice_unclassified_instance = [2, 1]
train_rf.classify_data_point(practice_unclassified_instance)

Node: [[0, 3, 0], [1, 3, 1], [2, 1, 2], [0, 3, 0], [1, 3, 1], [2, 1, 2]]

Node 1 with a feature index of 0 and a feature instance of 0: [[0, 3, 0], [0, 3, 0]]
Node 2 with a feature index of 0 and a feature instance of 0: [[1, 3, 1], [2, 1, 2], [1, 3, 1], [2, 1, 2]]
All classes in the dataset: [0, 1, 2, 0, 1, 2]

All unique labels in the dataset: [0, 1, 2]

Calculated entropy: 1.0

Node 1 with a feature index of 0 and a feature instance of 1: [[1, 3, 1], [1, 3, 1]]
Node 2 with a feature index of 0 and a feature instance of 1: [[0, 3, 0], [2, 1, 2], [0, 3, 0], [2, 1, 2]]
All classes in the dataset: [0, 1, 2, 0, 1, 2]

All unique labels in the dataset: [0, 1, 2]

Calculated entropy: 1.0

Node 1 with a feature index of 0 and a feature instance of 2: [[2, 1, 2], [2, 1, 2]]
Node 2 with a feature index of 0 and a feature instance of 2: [[0, 3, 0], [1, 3, 1], [0, 3, 0], [1, 3, 1]]
All classes in the dataset: [0, 1, 2, 0, 1, 2]

All unique labels in the dataset: [0, 1, 2]

Calculated entropy: 1